In [1]:
"""
Інтеграція: Knowledge Graph (networkx) + Production Rules + Frames (структури)
Приклад: система лікарських рослин з фреймами, що описують форми, дози, компоненти.
"""

import networkx as nx
from pprint import pprint
from typing import Dict, List

# ---------------------------
# 1) Knowledge Graph (G)
# ---------------------------
G = nx.DiGraph()

def add(rel, a, b):
    G.add_edge(a, b, relation=rel)

plants = [
    "Ромашка", "Календула", "Звіробій", "Подорожник", "Полин",
    "М’ята", "Валеріана", "Чебрець", "Алоє", "Евкаліпт"
]

for p in plants:
    add("is_a", p, "Лікарська_рослина")

relations = [
    ("Ромашка", "Протизапальна", "has_property"),
    ("Ромашка", "Ранозагоювальна", "has_property"),
    ("Ромашка", "Гастрит", "treats"),
    ("Ромашка", "Опіки", "treats"),
    ("Ромашка", "Алергічна_чутливість", "contraindicated_for"),

    ("Календула", "Протизапальна", "has_property"),
    ("Календула", "Антисептична", "has_property"),
    ("Календула", "Рани_та_порізи", "treats"),
    ("Календула", "Опіки", "treats"),

    ("Звіробій", "Антисептична", "has_property"),
    ("Звіробій", "Жовчогінна", "has_property"),
    ("Звіробій", "Проблеми_шлунку", "treats"),
    ("Звіробій", "Вагітність", "contraindicated_for"),

    ("Подорожник", "Ранозагоювальна", "has_property"),
    ("Подорожник", "Рани_та_порізи", "treats"),
    ("Подорожник", "Гастрит", "treats"),

    ("Полин", "Жовчогінна", "has_property"),
    ("Полин", "Протимікробна", "has_property"),
    ("Полин", "Проблеми_шлунку", "treats"),
    ("Полин", "Виразкова_хвороба", "contraindicated_for"),

    ("М’ята", "Спазмолітична", "has_property"),
    ("М’ята", "Заспокійлива", "has_property"),
    ("М’ята", "Нервове_напруження", "treats"),

    ("Валеріана", "Заспокійлива", "has_property"),
    ("Валеріана", "Нервове_напруження", "treats"),
    ("Валеріана", "Артеріальна_гіпертензія", "treats"),

    ("Чебрець", "Відхаркувальна", "has_property"),
    ("Чебрець", "Антисептична", "has_property"),
    ("Чебрець", "Кашель", "treats"),
    ("Чебрець", "Бронхіт", "treats"),

    ("Алоє", "Ранозагоювальна", "has_property"),
    ("Алоє", "Протизапальна", "has_property"),
    ("Алоє", "Опіки", "treats"),
    ("Алоє", "Рани_та_порізи", "treats"),

    ("Евкаліпт", "Протимікробна", "has_property"),
    ("Евкаліпт", "Відхаркувальна", "has_property"),
    ("Евкаліпт", "Бронхіт", "treats"),
    ("Евкаліпт", "ГРВІ", "treats"),
]

for a, b, r in relations:
    add(r, a, b)

# -- графові запити --
def get_properties(plant: str) -> List[str]:
    return [t for t in G.successors(plant) if G[plant][t]["relation"] == "has_property"]

def treats(plant: str) -> List[str]:
    return [t for t in G.successors(plant) if G[plant][t]["relation"] == "treats"]

def find_plants_for(condition: str) -> List[str]:
    return [p for p in G.predecessors(condition) if G[p][condition]["relation"] == "treats"]

# ---------------------------
# 2) Production Rules (інтерпретація)
# ---------------------------
plant_properties = {
    "Ромашка": ["Протизапальна", "Ранозагоювальна"],
    "Календула": ["Протизапальна", "Антисептична"],
    "Звіробій": ["Антисептична", "Жовчогінна"],
    "Подорожник": ["Ранозагоювальна"],
    "Полин": ["Жовчогінна", "Протимікробна"],
    "М’ята": ["Спазмолітична", "Заспокійлива"],
    "Валеріана": ["Заспокійлива"],
    "Чебрець": ["Відхаркувальна", "Антисептична"],
    "Алоє": ["Ранозагоювальна", "Протизапальна"],
    "Евкаліпт": ["Відхаркувальна", "Протимікробна"]
}

plant_contra = {
    "Ромашка": ["Алергічна_чутливість"],
    "Звіробій": ["Вагітність"],
    "Полин": ["Виразкова_хвороба"],
}

rules = [
    ("Протизапальна", "Рекомендація: застосовується при опіках та запаленнях тканин.", "indication"),
    ("Ранозагоювальна", "Рекомендація: застосовується при ранах, порізах та пошкодженнях шкіри.", "indication"),
    ("Антисептична", "Рекомендація: ефективна для знезараження та пригнічення бактерій.", "indication"),
    ("Заспокійлива", "Рекомендація: використовується при нервовому напруженні та стресі.", "indication"),
    ("Відхаркувальна", "Рекомендація: застосовується при кашлі та бронхіті.", "indication"),
    ("Жовчогінна", "Рекомендація: підтримує роботу печінки та травлення.", "indication"),
    ("Протимікробна", "Рекомендація: допомагає при інфекційних процесах.", "indication"),
    ("Спазмолітична", "Рекомендація: полегшує спазми шлунково-кишкового тракту.", "indication"),
]

def infer_recommendations(plant: str) -> List[Dict[str,str]]:
    if plant not in plant_properties:
        return [{"rule_prop": None, "conclusion": "Немає даних про цю рослину.", "type": "error"}]
    results = []
    for prop in plant_properties[plant]:
        for rule_prop, conclusion, typ in rules:
            if prop == rule_prop:
                results.append({"rule_prop": prop, "conclusion": conclusion, "type": typ})
    if plant in plant_contra:
        for contra in plant_contra[plant]:
            results.append({"rule_prop": "Contra", "conclusion": f"Увага: протипоказано при {contra}.", "type": "warning"})
    return results

def infer_recommendations_with_profile(plant: str, profile: Dict[str, str]) -> List[Dict[str,str]]:
    results = infer_recommendations(plant)
    if plant in plant_contra:
        user_flags = set()
        if profile.get("pregnancy"):
            user_flags.add("Вагітність")
        if profile.get("condition"):
            user_flags.add(profile.get("condition"))
        if profile.get("allergy_to_plant"):
            # якщо користувач вказав алергію на рослину категорично
            if profile.get("allergy_to_plant") == plant:
                user_flags.add("Алергічна_чутливість")
        for contra in plant_contra[plant]:
            if contra in user_flags:
                results.append({
                    "rule_prop": "Contra_Profile",
                    "conclusion": f"Критично: рослину не можна застосовувати через фактор '{contra}' у профілі користувача.",
                    "type": "critical"
                })
    return results

# ---------------------------
# 3) Frames (фрейми) - НОВА система
# ---------------------------
# Фрейм дає структуровану інформацію для кожної рослини.
plant_frames = {
    "Ромашка": {
        "forms": ["чай", "настоянка", "компрес", "мазь"],
        "typical_dose": {
            "чай": "1-2 ч.л. на чашку, 2-3 рази на день",
            "настоянка": "10-20 крапель 2 рази на день",
            "компрес": "зовнішньо, за потреби",
            "мазь": "тонкий шар 1-2 рази на день"
        },
        "active_components": ["апігенін", "хамазулен"],
        "best_for": ["Гастрит", "Опіки"],
        "notes": "Може викликати алергічну реакцію у чутливих осіб."
    },
    "Календула": {
        "forms": ["мазь", "настоянка", "полоскання"],
        "typical_dose": {"мазь": "зовнішньо 1-2 рази", "настоянка": "зовнішньо або внутрішньо за рецептом"},
        "active_components": ["каротиноїди", "флавоноїди"],
        "best_for": ["Рани_та_порізи", "Опіки"],
        "notes": "Добре для зовнішнього застосування."
    },
    "Звіробій": {
        "forms": ["настій", "масло"],
        "typical_dose": {"настій": "1 склянка на день"},
        "active_components": ["гіперицин", "гіперфорин"],
        "best_for": ["Проблеми_шлунку"],
        "notes": "Протипоказаний при вагітності; взаємодіє з деякими ліками (міжклітинна взаємодія)."
    },
    # ... інші фрейми (для скорочення наведено частково)
    "Подорожник": {
        "forms": ["компрес", "настій", "сироп"],
        "typical_dose": {"настій": "2-3 рази на день"},
        "active_components": ["слизи", "фітонциди"],
        "best_for": ["Рани_та_порізи", "Гастрит"],
        "notes": "Підходить для зовнішнього і внутрішнього застосування за показами."
    },
    "Алоє": {
        "forms": ["гель", "мазь", "сік"],
        "typical_dose": {"гель": "зовнішньо за потреби"},
        "active_components": ["полісахариди", "антоціани"],
        "best_for": ["Опіки", "Рани_та_порізи"],
        "notes": "Для зовнішнього застосування найчастіше."
    }
}

# Функція: вибір рекомендованої форми на основі умови та профілю
def choose_form_for_use(plant: str, condition: str, profile: Dict[str, str]) -> Dict[str, str]:
    frame = plant_frames.get(plant)
    if not frame:
        return {"recommended_form": None, "reason": "Немає фрейм-інформації для цієї рослини."}
    # Пріоритет: якщо форма в best_for відповідає умові — запропонувати зовнішню або внутрішню форму
    best = frame.get("best_for", [])
    # Умови, що вимагають зовнішнього використання (опіки, рани)
    external_conditions = {"Опіки", "Рани_та_порізи"}
    # Якщо умова є в external_conditions — обираємо зовнішню форму
    if condition in external_conditions:
        # знайти зовнішню форму у frame.forms (ґрубо припускаємо: мазь, компрес, гель)
        for f in ["мазь", "компрес", "гель", "сік"]:
            if f in frame["forms"]:
                return {"recommended_form": f, "reason": f"Умова {condition} потребує зовнішнього застосування; форма {f} з фрейму."}
    # Якщо умова у списку best_for — запропонувати першу доступну форму
    if condition in best:
        return {"recommended_form": frame["forms"][0], "reason": f"Рослина ефективна для {condition}; рекомендована форма: {frame['forms'][0]}."}
    # Якщо профіль — вагітність, обираємо більш м’яку форму або None
    if profile.get("pregnancy") and plant == "Звіробій":
        return {"recommended_form": None, "reason": "Протипоказано при вагітності."}
    # Інакше — загальна рекомендована форма (перша)
    return {"recommended_form": frame["forms"][0], "reason": "Загальна рекомендація згідно фрейму."}

# ---------------------------
# 4) Інтеграційні функції
# ---------------------------
def explain_plant_integrated(plant: str, profile: Dict[str,str]=None) -> Dict:
    if profile is None:
        profile = {}
    info = {
        "plant": plant,
        "class": "Лікарська_рослина" if "Лікарська_рослина" in [n for n in G.successors(plant)] else None,
        "properties": get_properties(plant),
        "treats": treats(plant),
        "rules_inference": infer_recommendations_with_profile(plant, profile),
        "frame": plant_frames.get(plant, {}),
    }
    return info

def recommend_for_condition(condition: str, profile: Dict[str,str]=None) -> Dict:
    """
    Комплексний сценарій: знайти рослини, інтерпретувати правила, застосувати фрейми для вибору форми.
    Повертає ранжований список кандидатів з поясненнями.
    """
    if profile is None:
        profile = {}
    plants = find_plants_for(condition)
    results = []
    for p in plants:
        infra = explain_plant_integrated(p, profile)
        # визначити чи є критичні протипоказання
        crits = [r for r in infra["rules_inference"] if r["type"] in ("critical", "warning")]
        # вибір форми через фрейм
        frame_choice = choose_form_for_use(p, condition, profile)
        # фінальна рекомендація (логіка правил + фрейм)
        if any(r["type"] == "critical" for r in infra["rules_inference"]):
            final = {"status": "NOT_RECOMMENDED", "reason": "Є критичні протипоказання у профілі."}
        else:
            final = {"status": "RECOMMENDED", "form": frame_choice["recommended_form"], "advice": frame_choice["reason"]}
        results.append({
            "plant": p,
            "properties": infra["properties"],
            "rules": infra["rules_inference"],
            "frame": infra["frame"],
            "final_decision": final
        })
    return {"condition": condition, "profile": profile, "candidates": results}

# ---------------------------
# 5) Демонстрація / Сценарії
# ---------------------------

def print_recommendation_summary(res: Dict):
    print(f"\n--- Рекомендації для умови: {res['condition']} ---")
    print(f"Профіль користувача: {res['profile']}")
    for c in res["candidates"]:
        print(f"\nРослина: {c['plant']}")
        print(f" Властивості: {c['properties']}")
        print(f" Правила (висновки):")
        for r in c["rules"]:
            print(f"  - [{r['type']}] {r['rule_prop']}: {r['conclusion']}")
        print(f" Фрейм (форми): {list(c['frame'].get('forms', []))[:3]} ...")
        fd = c["final_decision"]
        if fd["status"] == "NOT_RECOMMENDED":
            print(" Фінальне рішення: НЕ РЕКОМЕНДОВАНО —", fd["reason"])
        else:
            print(" Фінальне рішення: РЕКОМЕНДОВАНО")
            print("  Рекомендована форма:", fd.get("form"))
            print("  Порада:", fd.get("advice"))

# Сценарій 1: користувач запитує про гастрит, без вагітності
profile1 = {"pregnancy": False, "type": None}
res1 = recommend_for_condition("Гастрит", profile1)
print_recommendation_summary(res1)

# Сценарій 2: користувач запитує про гастрит та вказує вагітність=True
profile2 = {"pregnancy": True}
res2 = recommend_for_condition("Гастрит", profile2)
print_recommendation_summary(res2)

# Сценарій 3: користувач з алергією на ромашку питає про опіки
profile3 = {"allergy_to_plant": "Ромашка"}
res3 = recommend_for_condition("Опіки", profile3)
print_recommendation_summary(res3)



--- Рекомендації для умови: Гастрит ---
Профіль користувача: {'pregnancy': False, 'type': None}

Рослина: Ромашка
 Властивості: ['Протизапальна', 'Ранозагоювальна']
 Правила (висновки):
  - [indication] Протизапальна: Рекомендація: застосовується при опіках та запаленнях тканин.
  - [indication] Ранозагоювальна: Рекомендація: застосовується при ранах, порізах та пошкодженнях шкіри.
  - [warning] Contra: Увага: протипоказано при Алергічна_чутливість.
 Фрейм (форми): ['чай', 'настоянка', 'компрес'] ...
 Фінальне рішення: РЕКОМЕНДОВАНО
  Рекомендована форма: чай
  Порада: Рослина ефективна для Гастрит; рекомендована форма: чай.

Рослина: Подорожник
 Властивості: ['Ранозагоювальна']
 Правила (висновки):
  - [indication] Ранозагоювальна: Рекомендація: застосовується при ранах, порізах та пошкодженнях шкіри.
 Фрейм (форми): ['компрес', 'настій', 'сироп'] ...
 Фінальне рішення: РЕКОМЕНДОВАНО
  Рекомендована форма: компрес
  Порада: Рослина ефективна для Гастрит; рекомендована форма: компрес.